<a href="https://colab.research.google.com/github/flohmannjr/tensorflow_curso/blob/main/TensorFlow_MP2_SkimLit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TensorFlow Milestone Project: SkimLit

Este projeto tem como objetivo reproduzir o artigo **PubMed 200k RCT: a Dataset for Sequential Sentence Classification in Medical Abstracts**. https://arxiv.org/abs/1710.06071

O resumo de artigos científicos de Medicina costumam ser blocos de texto de difícil interpretação imediata. Este artigo teve como propósito criar um modelo para classificação sequencial de sentenças, onde, uma vez apresentado um resumo, dividirá o texto em cinco categorias: Background, Conclusions, Methods, Objective e Results (Contexto, Conclusões, Métodos, Objetivo e Resultados), facilitando sua interpretação.

PubMed 200k RCT utiliza a arquitetura do modelo do artigo **Neural Networks for Joint Sentence Classification in Medical Paper Abstracts**. https://arxiv.org/abs/1612.05251

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import random, re

from tensorflow.data import AUTOTUNE, Dataset
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.layers import Embedding, TextVectorization
from tensorflow.keras.layers import Dense, GlobalAveragePooling1D, GlobalMaxPool1D, Input
from tensorflow.keras.layers import Conv1D

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

## Setup

In [ ]:
pd.set_option('display.max_colwidth', None)

plt.rcParams['figure.figsize'] = [8, 5]
plt.rcParams['figure.dpi'] = 100

# plt.style.use('seaborn-darkgrid')
sns.set_style('darkgrid')

### Constantes

In [ ]:
SEMENTE = 2008193

COR = '#007f66'

DIRETORIO = '/content/pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign'
# DIRETORIO = '/content/pubmed-rct/PubMed_200k_RCT_numbers_replaced_with_at_sign'

# PubMed 200k RCT - Table 2
LIMITE_DICIONARIO = 68000
# LIMITE_DICIONARIO = 331000

LIMITE_CARACTERES = 100

# Neural Networks for Joint Sentence Classification - Figure 1
LIMITE_INCORPORADOR = 300
LIMITE_INCORPORADOR_CARACTERES = 25

LOTE_TAMANHO = 32

ENTRADA_FORMATO = (1,)
ENTRADA_TIPO = tf.string

ATIVACAO = 'relu'
ATIVACAO_SAIDA = 'softmax'

FILTROS = 64
NUCLEO_TAMANHO = 5
PREENCHIMENTO = 'same'

PERDA = 'categorical_crossentropy'
APRENDIZADO = 0.001
METRICAS = ['accuracy']

ITERACOES = 3

### Funções

In [ ]:
!wget https://raw.githubusercontent.com/flohmannjr/tensorflow_curso/main/funcoes.py

In [ ]:
from funcoes import avaliar_modelo, grafico_historico_por_iteracao

In [ ]:
def preprocessar_texto(arquivo):
    """
    Retorna uma lista de dicionários com o conteúdo das linhas do arquivo informado.

    Cada dicionário contém o código do resumo, o total de linhas, o número, a classe e o texto da linha.
    {codigo, total, numero, classe, texto}
    """

    with open(arquivo, 'r') as a: linhas_arquivo = a.readlines()
    linhas = ''  # Linhas do texto
    saida = []   # Lista de dicionários

    for linha in linhas_arquivo:
        if linha.startswith('###'):            # Verifica se a linha inicia com '###'.
            codigo = re.sub('\D+', '', linha)  # Extrai o código do resumo.
            linhas = ''                        # Limpa as linhas de texto.
        
        elif linha.isspace():                       # Verifica se a linha é vazia.
            linhas_separadas = linhas.splitlines()  # Separa linhas de texto.

            for linha_numero, linha_completa in enumerate(linhas_separadas):  # Itera sobre cada uma das linhas de texto.
                linha_dados = {}                                              # Dicionário com os dados da linha.
                linha_conteudo = linha_completa.split('\t')                   # Separa classe e texto.

                linha_dados['codigo'] = codigo                    # Código do resumo.
                linha_dados['total'] = len(linhas_separadas)      # Total de linhas.
                linha_dados['numero'] = linha_numero              # Número da linha.
                linha_dados['classe'] = linha_conteudo[0]         # Classe da linha.
                linha_dados['texto'] = linha_conteudo[1].lower()  # Texto da linha, em caixa baixa.

                saida.append(linha_dados)  # Acrescenta o dicionário à lista.

        else:
            linhas += linha  # Acrescenta a linha atual às linhas de texto.

    return saida

## Dados

In [ ]:
!git clone https://github.com/Franck-Dernoncourt/pubmed-rct.git

### Pré-processamento

In [ ]:
linhas_treino    = preprocessar_texto(f'{DIRETORIO}/train.txt')  # Dados de treino
linhas_validacao = preprocessar_texto(f'{DIRETORIO}/dev.txt')    # Dados de validação
linhas_teste     = preprocessar_texto(f'{DIRETORIO}/test.txt')   # Dados de teste

In [ ]:
df_treino    = pd.DataFrame(linhas_treino)
df_validacao = pd.DataFrame(linhas_validacao)
df_teste     = pd.DataFrame(linhas_teste)

### Verificação

In [ ]:
df_treino[:12]

In [ ]:
df_treino['classe'].value_counts()

In [ ]:
sns.histplot(data=df_treino, x='total', color=COR)

plt.xlabel('Quantidade de linhas')
plt.ylabel('Quantidade de resumos');

### Rótulos numéricos

#### One-hot

In [ ]:
codificador_onehot = OneHotEncoder(sparse=False)

rotulos_onehot_treino    = codificador_onehot.fit_transform(df_treino['classe'].to_numpy().reshape(-1, 1))
rotulos_onehot_validacao = codificador_onehot.transform(df_validacao['classe'].to_numpy().reshape(-1, 1))
rotulos_onehot_teste     = codificador_onehot.transform(df_teste['classe'].to_numpy().reshape(-1, 1))

In [ ]:
rotulos_onehot_treino

#### Inteiros

In [ ]:
codificador_int = LabelEncoder()

rotulos_int_treino    = codificador_int.fit_transform(df_treino['classe'].to_numpy())
rotulos_int_validacao = codificador_int.transform(df_validacao['classe'].to_numpy())
rotulos_int_teste     = codificador_int.transform(df_teste['classe'].to_numpy())

In [ ]:
rotulos_int_treino

In [ ]:
classes = codificador_int.classes_
classes

## Modelos

In [ ]:
avaliacoes = [None] * 6

### Modelo 0: TF-IDF Naive-Bayes

In [ ]:
modelo_nome = 'modelo_0_tfidf_naive_bayes'

modelo = Pipeline([('tfidf', TfidfVectorizer()),
                   ('clf', MultinomialNB())])

modelo.fit(df_treino['texto'], rotulos_int_treino)

In [ ]:
probabilidades = modelo.predict(df_validacao['texto'])

In [ ]:
previsoes = np.zeros((probabilidades.size, probabilidades.max() + 1))
previsoes[np.arange(probabilidades.size), probabilidades] = 1

In [ ]:
avaliacoes[0] = avaliar_modelo(rotulos_int_validacao, probabilidades, classes)

In [ ]:
avaliacoes[0]

### Preparação dos textos

In [ ]:
palavras_por_texto    = [len(texto.split()) for texto in df_treino['texto']]
palavras_por_texto_95 = int(np.percentile(palavras_por_texto, 95))  # Quantidade máxima de palavras para 95% dos textos.

In [ ]:
print(f"Média: {np.mean(palavras_por_texto)}")
print(f"Máximo: {np.max(palavras_por_texto)}")
print(f"95%: {palavras_por_texto_95}")

In [ ]:
sns.histplot(x=palavras_por_texto, color=COR)

plt.xlim(0, 60)

plt.xlabel('Quantidade de palavras')
plt.ylabel('Quantidade de textos');

#### Vetorização

In [ ]:
vetorizador = TextVectorization(max_tokens=LIMITE_DICIONARIO,
                                output_mode='int',
                                output_sequence_length=palavras_por_texto_95,
                                name='vetorizador')

vetorizador.adapt(df_treino['texto'])

In [ ]:
vetorizador.get_config()

#### Vocabulário

In [ ]:
vocabulario = vetorizador.get_vocabulary()

print(f"Tamanho do vocabulário: {len(vocabulario)}")
print(f"10 palavras mais comuns: {vocabulario[:10]}")
print(f"10 palavras menos comuns: {vocabulario[-10:]}")

#### Incorporação

In [ ]:
incorporador = Embedding(input_dim=len(vocabulario),
                         output_dim=LIMITE_INCORPORADOR,
                         mask_zero=True,
                         input_length=palavras_por_texto_95,
                         name='incorporador')

`mask_zero=True` otimiza o tratamento de preenchimento com valor zero nas camadas subsequentes e na função de perda.

Mais informações: https://stackoverflow.com/questions/47485216/how-does-mask-zero-in-keras-embedding-layer-work

In [ ]:
incorporador.get_config()

#### Verificação

In [ ]:
texto_aleatorio = random.choice(df_treino['texto'])

vetorizado = vetorizador([texto_aleatorio])
incorporado = incorporador(vetorizado)

print(texto_aleatorio)
print()
print(f"Quantidade de palavras: {len(texto_aleatorio.split())}")
print(f"Formato vetorizado: {vetorizado.shape}")
print(f"Formato incorporado: {incorporado.shape}")
print()
print(vetorizado)
print()
print(incorporado)

#### Criação de datasets

Fusão dos dados e rótulos em datasets para ganho de performance.

Better performance with the tf.data API: https://www.tensorflow.org/guide/data_performance

In [ ]:
# Datasets
dados_treino    = Dataset.from_tensor_slices((df_treino['texto'], rotulos_onehot_treino))
dados_validacao = Dataset.from_tensor_slices((df_validacao['texto'], rotulos_onehot_validacao))
dados_teste     = Dataset.from_tensor_slices((df_teste['texto'], rotulos_onehot_teste))

In [ ]:
dados_treino

In [ ]:
# Pré-buscas
dados_treino    = dados_treino.batch(LOTE_TAMANHO).prefetch(AUTOTUNE)
dados_validacao = dados_validacao.batch(LOTE_TAMANHO).prefetch(AUTOTUNE)
dados_teste     = dados_teste.batch(LOTE_TAMANHO).prefetch(AUTOTUNE)

In [ ]:
dados_treino

### Modelo 1: Conv1D com incorporação de palavras

In [ ]:
modelo_nome = 'modelo_1_conv1d_inc_palavras'

entradas = Input(shape=ENTRADA_FORMATO, dtype=ENTRADA_TIPO, name='camada_entrada')

camadas = vetorizador(entradas)
camadas = incorporador(camadas)

camadas = Conv1D(filters=FILTROS,
                 kernel_size=NUCLEO_TAMANHO,
                 activation=ATIVACAO,
                 padding=PREENCHIMENTO,
                 name='camada_convulacional')(camadas)

camadas = GlobalAveragePooling1D(name='agrupamento_media_global')(camadas)

saidas = Dense(len(classes), activation=ATIVACAO_SAIDA, name='camada_saida')(camadas)

modelo = Model(inputs=entradas, outputs=saidas, name=modelo_nome)

modelo.compile(loss=PERDA,
               optimizer=Adam(learning_rate=APRENDIZADO),
               metrics=METRICAS)

historico = modelo.fit(dados_treino,
                       epochs=ITERACOES,
                       validation_data=dados_validacao,
                       verbose=1)

In [ ]:
grafico_historico_por_iteracao(historico)

In [ ]:
probabilidades = modelo.predict(dados_validacao)
previsoes      = tf.argmax(probabilidades, axis=1)
avaliacoes[1]  = avaliar_modelo(rotulos_int_validacao, previsoes, classes)

In [ ]:
avaliacoes[1]

### Modelo 2: TFHub USE

In [ ]:
modelo_nome = 'modelo_2_use'

entradas = Input(shape=[], dtype=ENTRADA_TIPO, name='camada_entrada')  # shape=[] para entradas de tamanho variável

camadas = hub.KerasLayer(handle='https://tfhub.dev/google/universal-sentence-encoder/4',
                         trainable=False,
                         name='camada_use')(entradas)

camadas = Dense(LIMITE_INCORPORADOR, activation=ATIVACAO, name='camada_relu')(camadas)

saidas = Dense(len(classes), activation=ATIVACAO_SAIDA, name='camada_saida')(camadas)

modelo = Model(inputs=entradas, outputs=saidas, name=modelo_nome)

modelo.compile(loss=PERDA,
               optimizer=Adam(learning_rate=APRENDIZADO),
               metrics=METRICAS)

historico = modelo.fit(dados_treino,
                       epochs=ITERACOES,
                       validation_data=dados_validacao,
                       verbose=1)

In [ ]:
grafico_historico_por_iteracao(historico)

In [ ]:
probabilidades = modelo.predict(dados_validacao)
previsoes      = tf.argmax(probabilidades, axis=1)
avaliacoes[2]  = avaliar_modelo(rotulos_int_validacao, previsoes, classes)

In [ ]:
avaliacoes[2]

### Preparação dos caracteres

In [ ]:
caracteres_por_texto    = [len(texto) for texto in df_treino['texto']]
caracteres_por_texto_95 = int(np.percentile(caracteres_por_texto, 95))  # Quantidade máxima de caracteres para 95% dos textos.

In [ ]:
print(f"Média: {np.mean(caracteres_por_texto)}")
print(f"Máximo: {np.max(caracteres_por_texto)}")
print(f"95%: {caracteres_por_texto_95}")

In [ ]:
sns.histplot(x=caracteres_por_texto, color=COR)

plt.xlim(0, 300)

plt.xlabel('Quantidade de caracteres')
plt.ylabel('Quantidade de textos');

#### Listas de caracteres

In [ ]:
lista_caracteres_treino    = [" ".join(list(texto)) for texto in df_treino['texto']]
lista_caracteres_validacao = [" ".join(list(texto)) for texto in df_validacao['texto']]
lista_caracteres_teste     = [" ".join(list(texto)) for texto in df_teste['texto']]

#### Vetorização

In [ ]:
vetorizador_caracteres = TextVectorization(max_tokens=LIMITE_CARACTERES,
                                           output_mode='int',
                                           output_sequence_length=caracteres_por_texto_95,
                                           name='vetorizador_caracteres')

vetorizador_caracteres.adapt(lista_caracteres_treino)

In [ ]:
vetorizador_caracteres.get_config()

#### Vocabulário

In [ ]:
vocabulario_caracteres = vetorizador_caracteres.get_vocabulary()

print(f"Tamanho do vocabulário: {len(vocabulario_caracteres)}")
print(f"10 caracteres mais comuns: {vocabulario_caracteres[:10]}")
print(f"10 caracteres menos comuns: {vocabulario_caracteres[-10:]}")

#### Incorporação

In [ ]:
incorporador_caracteres = Embedding(input_dim=len(vocabulario_caracteres),
                                    output_dim=LIMITE_INCORPORADOR_CARACTERES,
                                    mask_zero=True,
                                    input_length=caracteres_por_texto_95,
                                    name='incorporador_caracteres')

In [ ]:
incorporador_caracteres.get_config()

#### Verificação

In [ ]:
texto_aleatorio = random.choice(lista_caracteres_treino)

vetorizado = vetorizador_caracteres([texto_aleatorio])
incorporado = incorporador_caracteres(vetorizado)

print(texto_aleatorio)
print()
print(f"Quantidade de caracteres: {len(texto_aleatorio.split())}")
print(f"Formato vetorizado: {vetorizado.shape}")
print(f"Formato incorporado: {incorporado.shape}")
print()
print(vetorizado)
print()
print(incorporado)

#### Criação de datasets

In [ ]:
# Datasets
caracteres_treino    = Dataset.from_tensor_slices((lista_caracteres_treino, rotulos_onehot_treino))
caracteres_validacao = Dataset.from_tensor_slices((lista_caracteres_validacao, rotulos_onehot_validacao))
caracteres_teste     = Dataset.from_tensor_slices((lista_caracteres_teste, rotulos_onehot_teste))

In [ ]:
caracteres_treino

In [ ]:
# Pré-buscas
caracteres_treino    = caracteres_treino.batch(LOTE_TAMANHO).prefetch(AUTOTUNE)
caracteres_validacao = caracteres_validacao.batch(LOTE_TAMANHO).prefetch(AUTOTUNE)
caracteres_teste     = caracteres_teste.batch(LOTE_TAMANHO).prefetch(AUTOTUNE)

In [ ]:
caracteres_treino

### Modelo 3: Conv1D com incorporação de caracteres

In [ ]:
modelo_nome = 'modelo_3_conv1d_inc_caracteres'

entradas = Input(shape=ENTRADA_FORMATO, dtype=ENTRADA_TIPO, name='camada_entrada')

camadas = vetorizador_caracteres(entradas)
camadas = incorporador_caracteres(camadas)

camadas = Conv1D(filters=FILTROS,
                 kernel_size=NUCLEO_TAMANHO,
                 activation=ATIVACAO,
                 padding=PREENCHIMENTO,
                 name='camada_convulacional')(camadas)

camadas = GlobalMaxPool1D(name='agrupamento_maximo_global')(camadas)

saidas = Dense(len(classes), activation=ATIVACAO_SAIDA, name='camada_saida')(camadas)

modelo = Model(inputs=entradas, outputs=saidas, name=modelo_nome)

modelo.compile(loss=PERDA,
               optimizer=Adam(learning_rate=APRENDIZADO),
               metrics=METRICAS)

historico = modelo.fit(caracteres_treino,
                       epochs=ITERACOES * 2,
                       validation_data=caracteres_validacao,
                       verbose=1)

In [ ]:
grafico_historico_por_iteracao(historico)

In [ ]:
probabilidades = modelo.predict(caracteres_validacao)
previsoes      = tf.argmax(probabilidades, axis=1)
avaliacoes[3]  = avaliar_modelo(rotulos_int_validacao, previsoes, classes)

In [ ]:
avaliacoes[3]

### Modelo 4: Camada de incorporação híbrida